In [ ]:
#http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2odeeplearningestimator
import h2o
import pandas as pd
from h2o.estimators import H2ODeepLearningEstimator
from statistics import mean
import matplotlib.pyplot as plt
import time
import numpy as np
import scipy.stats as st

#h2o.cluster().shutdown()
h2o.init()
#h2o.cluster().show_status();
irisData = h2o.import_file("/Users/sukhwindersingh/Documents/study/sem2/PDC/iris/iris.data");
votesData = h2o.import_file("/Users/sukhwindersingh/Documents/study/sem2/PDC/votes/house-votes-84.data");
sonarData = h2o.import_file("/Users/sukhwindersingh/Documents/study/sem2/PDC/sonar/sonar.all-data");
#irisData.describe(chunk_summary=True);
#sonarData.describe(chunk_summary=True);
#votesData.describe(chunk_summary=True);

In [ ]:
print ('All  :', votesData.col_names)
print ('P  :', votesData.col_names[1:16])
print ('R  :', votesData.col_names[0])

In [ ]:
print ('ALL  :', irisData.col_names)
print ('P  :', irisData.col_names[0:4])
print ('R  :', irisData.col_names[4])
print ('----------------------------------')
print ('All  :', votesData.col_names)
print ('P  :', votesData.col_names[1:16])
print ('R  :', votesData.col_names[0])
print ('----------------------------------')
print ('All  :', sonarData.columns)
print ('P  :', sonarData.columns[:-1])
print ('R  :', sonarData.columns[60])


In [ ]:

    
Dict = {}
# Adding elements one at a time
Dict[0] = [0.15077023562772254, 0.21055075694250958]
Dict[1] = [0.9732789440345223, 0.027027027027026973]
Dict[2] = [5,5,5,5]  
print('fff',dictAvg(Dict))

In [ ]:
def calculate_confidenceInterval(ds,confidence):
    n = len(ds)
    m = np.mean(ds)
    std_err = st.sem(ds)
    std_dev= np.std(ds)
    moe = std_err * st.t.ppf((1 + confidence) / 2., n - 1) #marginOfError
    #print(" len",n," mean",m," s_err",std_err," s_dev",std_dev," moe",moe);
    return m,moe;

def plotGivenData(m,t,y):
    fig1, ax1 = plt.subplots()
    ax1.set_title(t)
    ax1.set_ylabel(y)
    ax1.set_xlabel('Method_folds[1]')
    ax1.boxplot(m.values(),labels=m.keys())

def calculateMPCE(dataSet,predictorColumns,responseColumn,dataSetName):
    map_train_error={}
    map_test_error={}
    map_time={}
    list_train_error=[]
    list_test_error=[]
    activations =["rectifier", "tanh"]
    folds = [0,5,10]
    for activation in activations:    
        for fold in folds:
            string_model = activation  +"_"+  str(fold)
            map_train_error[string_model]=[];
            map_test_error[string_model]=[];
            map_time[string_model]=[];
            for i in range(0,10):      
                dl = H2ODeepLearningEstimator(hidden=[5,20,100],
                                           epochs=40,
                                           activation=activation,
                                           nfolds=fold)
                if fold == 0:
                    trainData, testData = dataSet.split_frame(ratios=[.8])
                    t0 = time.time()
                    dl.train(x=predictorColumns,
                              y=responseColumn,
                              training_frame=trainData,
                              validation_frame=testData)
                    t0 = time.time() - t0
                    if(dataSetName=='Sonar'):
                        map_test_error[string_model].append(dl.mean_per_class_error(valid=True)[0][0])
                    else:
                        map_test_error[string_model].append(dl.mean_per_class_error(valid=True))
                    list_test_error.append(dl.mean_per_class_error(valid=True))
                else:
                    t0 = time.time()
                    dl.train(x=predictorColumns,
                         y=responseColumn,
                         training_frame=dataSet)
                    t0 = time.time() - t0
                    list_test_error.append(dl.mean_per_class_error(xval=True))
                    if(dataSetName=='Sonar'):
                        map_test_error[string_model].append(dl.mean_per_class_error(train=True)[0][0])
                    else:    
                        map_test_error[string_model].append(dl.mean_per_class_error(xval=True))
                list_train_error.append(dl.mean_per_class_error(train=True))
                if(dataSetName=='Sonar'):
                    map_train_error[string_model].append(dl.mean_per_class_error(train=True)[0][0])
                else:    
                    map_train_error[string_model].append(dl.mean_per_class_error(train=True))
                map_time[string_model].append(t0)
    rect_m = []
    rect_e = []
    tanh_m=[]
    tanh_e=[]
    for k, v in map_test_error.items():
        if(k.startswith('r')):
#             print(k," :  ",calculate_confidenceInterval(v,0.95));
            m,i=calculate_confidenceInterval(v,0.95)
            rect_m.append(m)
            rect_e.append(i)
        else:
#             print(k," :  ",calculate_confidenceInterval(v,0.95));
            m,i=calculate_confidenceInterval(v,0.95)
            tanh_m.append(m)
            tanh_e.append(i)
  
    plotGivenData(map_test_error,'Mean Per Class Error\nTest results for '+dataSetName+' dataset' , 'classification_error[1]')
    plotGivenData(map_train_error,'Mean Per Class Error\nTrain results for '+dataSetName+' dataset' , 'classification_error[1]')
    plotGivenData(map_time,'Time results for '+dataSetName+' dataset' , 'Time[1]')


    
#     https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51
#     https://philschatz.com/statistics-book/contents/m46920.html

    
calculateMPCE(irisData,irisData.col_names[0:4],irisData.col_names[4],'IRIS')
# calculateMPCE(votesData,votesData.col_names[1:16],votesData.columns[0],'Voter')
# calculateMPCE(sonarData,sonarData.col_names[0:60],sonarData.col_names[60],'Sonar')


In [ ]:
import numpy as np
import scipy.stats as st
import statistics
def calculate_confidenceInterval(ds,confidence):
    n = len(ds)
    m = np.mean(ds)
    std_err = st.sem(ds)
    std_dev= np.std(ds)
#     n = 50
#     m = 70
#     std_err = 0.7071
#     std_dev= 5
    moe = std_err * st.t.ppf((1 + confidence) / 2., n - 1) #marginOfError
    print(" len",n," mean",m," s_err",std_err," s_dev",std_dev," moe",moe);
    return m,moe;

# dataSet=[600,470,170,430,300]
dataSet=[10,20,30,40]
calculate_confidenceInterval(dataSet,0.95)
# https://www.mathsisfun.com/data/confidence-interval.html
# https://ncalculators.com/statistics/standard-error-calculator.htm


In [ ]:
 
    np.random.seed(10)
    data =  [0,5,10,3]
    
    print(" data ", data);
    
#     np.random.seed(19680801)

#         # fake up some data
#     spread = 10
#     center = 30 * 50
#     flier_high = 20  + 100
#     flier_low = 20  -100
#     data = np.concatenate((spread, center, flier_high, flier_low))
#     print(" center ", center);
#     print(" spread ", spread);   
#     print(" flier_high ",  flier_high );
#     print(" flier_low ", flier_low);
    fig1, ax1 = plt.subplots()
    ax1.set_title('Basic Plot')
    ax1.boxplot(data, showfliers=False)

In [ ]:
import scipy
def mean_confidence_interval(data, confidence):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
    return m, h

dataSet=[10,20,30,40]
print(mean_confidence_interval(dataSet,0.90))

In [ ]:
#for seq in list(range(6)):
print(list(range(6)))
# Generate predictions on a test set (if necessary):
#pred = dl.predict(testIRIS)
#print(pred)
#print(dl.auc)
#print(dl.mse)
# print(irisData['class'].categories())
# print(irisData.col_names)
# print(len(irisData.col_names))
   # adaptive_rate=True
            #train_samples_per_iteration=-1,
            #reproducible=True,
            #single_node_mode=False,
            #balance_classes=False,
            #force_load_balance=False
            # Eval performance:
#perf = dl.model_performance(testIRIS)
#print(perf)
# list_iris= []
# list_iris.append(dl.mse(valid=True))
# list_iris.append(dl.mse(valid=True))
# print(list_iris)

In [ ]:
for i in range(5):
    print(i)


In [ ]:
    xpoints_1 = list(map_train_error.keys())
    ypoints_1 = list(map_train_error.values())
    xpoints_2 = list(map_test_error.keys())
    ypoints_2 = list(map_test_error.values())
   
    plt.plot(xpoints_1, ypoints_1,label = "Train")
    plt.plot(xpoints_2, ypoints_2,label = "Test") 
    plt.legend()
    plt.xlabel('activation_folds')
    plt.ylabel('classification_error[1]')
    #plt.title('NAMEEEE')
    plt.show()
    